In [2]:
!pip install sqlalchemy==1.4.46
!pip install pandasql
!pip install geopy
!pip install -U kaleido
import pandas as pd
import pandasql as ps
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/drive')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 24.4 MB/s eta 0:00:00
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.29
    Uninstalling SQLAlchemy-2.0.29:
      Successfully uninstalled SQLAlchemy-2.0.29
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.4.46 which is incompatible.
  Preparing metadata (setup.py) ... done
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26771 sha256=b4f86c23ac12692630eaadc02e061911c3a81a2beae2c5b503361c6cd4a95f15
  Stored in directory: /root/.cache/pip/wheels/e9/bc/3a/8434bdcccf5779e72894a9b24fecbdcaf97940607eaf4bcdf9
Successfully built pandasql
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 8.0 MB/s eta 0:00:00
Mounted at /content/drive


## Crime data and Subarea data

In [80]:
crime_df = pd.read_csv("/content/drive/MyDrive/Crime_Data_from_2020_to_Present.csv")
crime_df

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,...,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
0,190326475,03/01/2020 12:00:00 AM,03/01/2020 12:00:00 AM,2130,7,Wilshire,784,1,510,VEHICLE - STOLEN,...,AA,Adult Arrest,510.0,998.0,NaN,NaN,1900 S LONGWOOD AV,NaN,34.0375,-118.3506
1,200106753,02/09/2020 12:00:00 AM,02/08/2020 12:00:00 AM,1800,1,Central,182,1,330,BURGLARY FROM VEHICLE,...,IC,Invest Cont,330.0,998.0,NaN,NaN,1000 S FLOWER ST,NaN,34.0444,-118.2628
2,200320258,11/11/2020 12:00:00 AM,11/04/2020 12:00:00 AM,1700,3,Southwest,356,1,480,BIKE - STOLEN,...,IC,Invest Cont,480.0,NaN,NaN,NaN,1400 W 37TH ST,NaN,34.0210,-118.3002
3,200907217,05/10/2023 12:00:00 AM,03/10/2020 12:00:00 AM,2037,9,Van Nuys,964,1,343,SHOPLIFTING-GRAND THEFT ($950.01 & OVER),...,IC,Invest Cont,343.0,NaN,NaN,NaN,14000 RIVERSIDE DR,NaN,34.1576,-118.4387
4,220614831,08/18/2022 12:00:00 AM,08/17/2020 12:00:00 AM,1200,6,Hollywood,666,2,354,THEFT OF IDENTITY,...,IC,Invest Cont,354.0,NaN,NaN,NaN,1900 TRANSIENT,NaN,34.0944,-118.3277
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
910702,242106032,02/20/2024 12:00:00 AM,02/17/2024 12:00:00 AM,2330,21,Topanga,2103,1,420,THEFT FROM MOTOR VEHICLE - PETTY ($950 & UNDER),...,IC,Invest Cont,420.0,NaN,NaN,NaN,22700 NAPA ST,NaN,34.2260,-118.6214
910703,241604246,01/08/2024 12:00:00 AM,01/08/2024 12:00:00 AM,800,16,Foothill,1601,2,625,OTHER ASSAULT,...,IC,Invest Cont,625.0,NaN,NaN,NaN,VAUGHN ST,DE GARMO AV,34.2865,-118.4161
910704,242004546,01/16/2024 12:00:00 AM,01/16/2024 12:00:00 AM,1510,20,Olympic,2013,2,624,BATTERY - SIMPLE ASSAULT,...,IC,Invest Cont,624.0,NaN,NaN,NaN,HOBART BL,3RD ST,34.0690,-118.3054
910705,240705409,02/02/2024 12:00:00 AM,01/31/2024 12:00:00 AM,30,7,Wilshire,784,1,510,VEHICLE - STOLEN,...,IC,Invest Cont,510.0,NaN,NaN,NaN,2300 S SYCAMORE AV,NaN,34.0360,-118.3527


In [81]:
crime_df.describe(include='all')
crime_df.shape

(910707, 28)

### Seperate R2 to R6
Tables on Area, Crime, Premis, Weapon and Status.

In [82]:
# seperate R2
R2_area = crime_df[['AREA','AREA NAME']].drop_duplicates()
R2_area_sorted = R2_area.sort_values(by='AREA').reset_index(drop=True)

R1_main = crime_df.drop(['AREA NAME'], axis=1)

print(R2_area_sorted)
print(R2_area_sorted.shape) # 21 different areas

    AREA    AREA NAME
0      1      Central
1      2      Rampart
2      3    Southwest
3      4   Hollenbeck
4      5       Harbor
5      6    Hollywood
6      7     Wilshire
7      8      West LA
8      9     Van Nuys
9     10  West Valley
10    11    Northeast
11    12  77th Street
12    13       Newton
13    14      Pacific
14    15  N Hollywood
15    16     Foothill
16    17   Devonshire
17    18    Southeast
18    19      Mission
19    20      Olympic
20    21      Topanga
(21, 2)


In [83]:
# get subarea information from LA_area table
area_df = pd.read_csv("/content/drive/MyDrive/LA_area.csv", header=None)

# explode area - subarea
area_df[1] = area_df[1].str.split(',')
area_df = area_df.explode(1).reset_index()
area_df.rename(columns={'index': 'AREA', 0: 'AREA NAME', 1: 'SUBAREA NAME'}, inplace=True)
area_df['AREA'] = area_df['AREA'] + 1

# Convert all characters in the 'AREA NAME' column to lowercase, then capitalize the first letter
area_df['AREA NAME'] = area_df['AREA NAME'].str.lower().str.capitalize()

In [84]:
merged_df = pd.merge(R2_area_sorted, area_df[['AREA','SUBAREA NAME']], on='AREA', how='inner')
merged_df

,AREA,AREA NAME,SUBAREA NAME
0,1,Central,Bunker Hill/Historic Core
1,1,Central,Central City East
2,1,Central,Chinatown
3,1,Central,Civic Center
4,1,Central,Downtown Los Angeles
...,...,...,...
199,20,Olympic,Arlington Heights
200,21,Topanga,Canoga Park
201,21,Topanga,West Hills
202,21,Topanga,Winnetka


In [85]:
merged_df.to_csv('/content/drive/MyDrive/550 CrimeLA data/R2_area.csv', index=False)

In [86]:
# seperate R3
R3_crime = crime_df[['Crm Cd','Crm Cd Desc']].drop_duplicates()
R3_crime_sorted = R3_crime.sort_values(by='Crm Cd').reset_index(drop=True)
R3_crime_sorted.to_csv('/content/drive/MyDrive/550 CrimeLA data/R3_crime.csv', index=False)

R1_main = R1_main.drop(['Crm Cd Desc'], axis=1)

print(R3_crime_sorted)
print(R3_crime_sorted.shape) # 139 different crimes

     Crm Cd                                        Crm Cd Desc
0       110                                  CRIMINAL HOMICIDE
1       113                            MANSLAUGHTER, NEGLIGENT
2       121                                     RAPE, FORCIBLE
3       122                                    RAPE, ATTEMPTED
4       210                                            ROBBERY
..      ...                                                ...
134     949                                    ILLEGAL DUMPING
135     950  DEFRAUDING INNKEEPER/THEFT OF SERVICES, OVER $...
136     951  DEFRAUDING INNKEEPER/THEFT OF SERVICES, $950 &...
137     954                                       CONTRIBUTING
138     956            LETTERS, LEWD  -  TELEPHONE CALLS, LEWD

[139 rows x 2 columns]
(139, 2)


In [87]:
# seperate R4
R4_premis = crime_df[['Premis Cd','Premis Desc']].drop_duplicates().dropna()
R4_premis_sorted = R4_premis.sort_values(by='Premis Cd').reset_index(drop=True)
R4_premis_sorted.to_csv('/content/drive/MyDrive/550 CrimeLA data/R4_premis.csv', index=False)

R1_main = R1_main.drop(['Premis Desc'], axis=1)

print(R4_premis_sorted)
print(R4_premis_sorted.shape) # 307 different premis (places)

     Premis Cd                         Premis Desc
0        101.0                              STREET
1        102.0                            SIDEWALK
2        103.0                               ALLEY
3        104.0                            DRIVEWAY
4        105.0             PEDESTRIAN OVERCROSSING
..         ...                                 ...
302      967.0         MTA - GOLD LINE - CHINATOWN
303      968.0   MTA - GOLD LINE - LINCOLN/CYPRESS
304      969.0       MTA - GOLD LINE - HERITAGE SQ
305      970.0  MTA - GOLD LINE - SOUTHWEST MUSEUM
306      971.0     MTA - GOLD LINE - HIGHLAND PARK

[307 rows x 2 columns]
(307, 2)


In [88]:
# seperate R5
R5_weapon = crime_df[['Weapon Used Cd','Weapon Desc']].drop_duplicates().dropna()
R5_weapon_sorted = R5_weapon.sort_values(by='Weapon Used Cd').reset_index(drop=True)
R5_weapon_sorted.to_csv('/content/drive/MyDrive/550 CrimeLA data/R5_weapon.csv', index=False)

R1_main = R1_main.drop(['Weapon Desc'], axis=1)

print(R5_weapon_sorted)
print(R5_weapon_sorted.shape) # 79 weapons

    Weapon Used Cd                 Weapon Desc
0            101.0                    REVOLVER
1            102.0                    HAND GUN
2            103.0                       RIFLE
3            104.0                     SHOTGUN
4            105.0     SAWED OFF RIFLE/SHOTGUN
..             ...                         ...
74           512.0           MACE/PEPPER SPRAY
75           513.0                    STUN GUN
76           514.0                   TIRE IRON
77           515.0           PHYSICAL PRESENCE
78           516.0  DOG/ANIMAL (SIC ANIMAL ON)

[79 rows x 2 columns]
(79, 2)


In [89]:
# seperate R6
R6_status = crime_df[['Status','Status Desc']].drop_duplicates().reset_index(drop=True)
R6_status.to_csv('/content/drive/MyDrive/550 CrimeLA data/R6_status.csv', index=False)

R1_main = R1_main.drop(['Status Desc'], axis=1)

print(R6_status)
print(R6_status.shape) # 6 different status

  Status   Status Desc
0     AA  Adult Arrest
1     IC   Invest Cont
2     JA    Juv Arrest
3     AO   Adult Other
4     JO     Juv Other
5     CC           UNK
(6, 2)


### Edit main crime table - R1
Drop unimportant columns, and aggregate date column.

In [92]:
# check R1
print(R1_main.shape)

# drop exact location info and sub-area info to generalize to area level
R1_crime = R1_main.drop(columns=['LOCATION', 'Cross Street', 'LAT', 'LON'])
R1_crime = R1_crime.drop(columns=['Rpt Dist No', 'Part 1-2'])
R1_crime = R1_crime.drop(columns=['Vict Age', 'Vict Sex', 'Vict Descent'])
R1_crime = R1_crime.drop(columns=['Date Rptd'])

(910707, 23)


In [93]:
R1_crime = R1_crime.drop(columns=['Mocodes'])

In [94]:
# Check data types of 'DATE OCC' columns
date_occ_dtype = R1_crime['DATE OCC'].dtype

# Display the data types
print(f"'DATE OCC' data type: {date_occ_dtype}")

'DATE OCC' data type: object


In [95]:
# Convert 'DATE OCC' to datetime
R1_crime['DATE OCC'] = pd.to_datetime(R1_crime['DATE OCC'], format='%m/%d/%Y %I:%M:%S %p')

# Extract date components for 'DATE OCC'
R1_crime['Date Occ'] = R1_crime['DATE OCC'].dt.date
R1_crime['Time Occ'] = R1_crime['TIME OCC'].apply(lambda x: "{:02d}:{:02d}".format(x // 100, x % 100))

In [106]:
R1_crime = R1_crime.drop(columns=['DATE OCC','TIME OCC'])

In [97]:
# Find rows where 'Crime Cd' is not equal to 'Crime Cd 1'
different_row = R1_crime.loc[R1_crime['Crm Cd'] != R1_crime['Crm Cd 1']]

if not different_row.empty:
    print(len(different_row))
else:
    print("All rows have 'Crime Cd' equal to 'Crime Cd 1'.")

1879


In [98]:
# Find rows where 'Crime Cd 2' is NaN
nan_row = R1_crime.loc[R1_crime['Crm Cd 2'].isna()]
if not nan_row.empty:
    print(len(nan_row))


844372


In [99]:
# Find rows where 'Crime Cd 3' is NaN
nan_row = R1_crime.loc[R1_crime['Crm Cd 3'].isna()]
if not nan_row.empty:
    print(len(nan_row))

908470


In [104]:
# Not much valid info, so we drop detailed Crm Code
R1_crime = R1_crime.drop(columns=['Crm Cd 1','Crm Cd 2','Crm Cd 3','Crm Cd 4'])

In [107]:
print(R1_crime.columns)

Index(['DR_NO', 'AREA', 'Crm Cd', 'Premis Cd', 'Weapon Used Cd', 'Status',
       'Date Occ', 'Time Occ'],
      dtype='object')


In [108]:
R1_crime.to_csv('/content/drive/MyDrive/550 CrimeLA data/R1_crime.csv', index=False)

## Airbnb Listing data

In [29]:
airbnb_df = pd.read_csv("/content/drive/MyDrive/listings.csv")

In [34]:
airbnb_df = airbnb_df.loc[airbnb_df['neighbourhood_group'] == 'City of Los Angeles']
airbnb_df = airbnb_df.drop(columns=['neighbourhood_group','latitude','longitude','last_review','reviews_per_month','calculated_host_listings_count','number_of_reviews_ltm','license'])

### parse name

In [35]:
def parse_entry(entry):
    parts = entry.split('·')
    name = parts[0].strip() if len(parts) > 0 else None
    star = parts[1].strip().replace('★', '') if len(parts) > 1 else None
    type_ = '·'.join(parts[2:]).strip() if len(parts) > 2 else None
    return pd.Series([name, star, type_])

In [38]:
# Parsing the 'entry' column into new columns 'name', 'star', and 'type'
airbnb_df[['airbnb_name', 'star', 'type']] = airbnb_df['name'].apply(parse_entry)
airbnb_df = airbnb_df.drop(columns=['name'])

### pasre room type

In [60]:
# Printing all unique 'type' values
unique_types = airbnb_df['type'].unique()
print(unique_types)

['1 bedroom · 1 bed · 1 shared bath' '1 bedroom · 2 beds · 1 private bath'
 '1 bedroom · 1 bed · 1 bath' '1 bedroom · 2 beds · 1 bath'
 '1 bedroom · 1 bed · 1.5 shared baths' '1 bedroom · 2 beds · 2 baths'
 'Studio · 2 beds · 1 bath' '3 bedrooms · 4 beds · 2 baths'
 '2 bedrooms · 10 beds · 1 bath' '1 bedroom · 3 beds · 1 bath'
 'Studio · 1 bath' '3 bedrooms · 3 beds · 2 baths'
 '4 bedrooms · 5 beds · 3 baths' '2 bedrooms · 2 beds · 1 bath'
 '3 bedrooms · 1 bed · 2 baths' '1 bedroom · 1 bed · 1 private bath'
 '1 bed · 1 bath' 'Studio · 1 bed · 1 bath'
 '3 bedrooms · 3 beds · 2.5 baths' '2 bedrooms · 2 beds · 1.5 baths'
 '2 bedrooms · 3 beds · 1 bath' '1 bedroom · 2 beds · 1.5 shared baths'
 '1 bedroom · 1 bed · 11 shared baths' '2 bedrooms · 1 bed · 1 bath'
 '1 bedroom · 2 beds · 1 shared bath' 'Studio · 1 bed · 11 shared baths'
 '2 bedrooms · 1 bed · 11 shared baths' '2 bedrooms · 2 beds · 2 baths'
 '1 bedroom · 2 beds · 1.5 baths' '5 bedrooms · 9 beds · 3 baths'
 '2 bedrooms · 3 beds 

In [69]:
def parse_type(entry):
    # Check if entry is None and return default values
    if entry is None:
      return pd.Series([None, None, None])

    num_bedrooms = None
    num_beds = None
    num_baths = None

    # Splitting the entry into parts for flexible analysis
    parts = entry.lower().split('·')

    for part in parts:
        if 'bedroom' in part or 'studio' in part:
            if 'studio' in part:
                num_bedrooms = 0  # Studios count as 0 bedrooms
            else:
                # Extract number of bedrooms
                num_bedrooms = part.strip().split()[0]

        elif 'bed' in part:
            # Extract number of beds
            num_beds = part.strip().split()[0]

        elif 'bath' in part:
            if 'half-bath' in part:
                num_baths = 0.5  # Handling half-baths specifically
            else:
                # Extract number of baths
                num_baths = part.strip().split()[0]

    return pd.Series([num_bedrooms, num_beds, num_baths])

# Example of how to use the function with a DataFrame column
# airbnb_df[['bedrooms', 'beds', 'baths']] = airbnb_df['type'].apply(parse_type_flexible)


In [70]:
airbnb_df[['bedrooms', 'beds', 'baths']] = airbnb_df['type'].apply(parse_type)

In [76]:
airbnb_df = airbnb_df.drop(columns=['type'])

In [73]:
nan_count = airbnb_df['bedrooms'].isna().sum()
print("Number of NaN values in 'bedrooms':", nan_count)

Number of NaN values in 'bedrooms': 9787


In [75]:
# Assuming the 'bedrooms' column has been added to airbnb_df and might contain NaN values

# Filter rows where 'bedrooms' is NaN and select the 'type' column
types_with_nan_bedrooms = airbnb_df[airbnb_df['bedrooms'].isna()]['type']

# Print out these types
print(types_with_nan_bedrooms)

44               1 bed · 1 bath
118          3 beds · 2.5 baths
133                        None
152              1 bed · 1 bath
157      1 bed · 1 private bath
                  ...          
42321            1 bed · 1 bath
42322          8 beds · 3 baths
42326            1 bed · 1 bath
43037         1 bed · Half-bath
44102           2 beds · 1 bath
Name: type, Length: 9787, dtype: object


In [78]:
airbnb_df

,id,host_id,host_name,neighbourhood,room_type,price,minimum_nights,number_of_reviews,availability_365,airbnb_name,star,bedrooms,beds,baths
1,2708,3008,Chas.,Hollywood,Private room,94,30,40,271,Rental unit in Los Angeles,4.85,1,1,1
3,63416,309512,Vincenzo,Hollywood Hills,Private room,220,2,161,0,Home in Hollywood,4.99,1,2,1
6,5728,9171,Sanni,Del Rey,Private room,55,30,316,120,Tiny home in Los Angeles,4.78,1,1,1
8,5843,9171,Sanni,Del Rey,Private room,60,30,128,179,Home in Los Angeles,4.63,1,1,1
9,6033,11619,Sarah,Woodland Hills,Entire home/apt,89,30,20,0,Bungalow in Woodland Hills,4.38,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45564,1036469508539912905,446661785,Lee,Encino,Entire home/apt,1000,3,0,35,Home in Los Angeles,New,4,5,4
45576,1037393173655149838,426737299,Avsar,Hollywood,Entire home/apt,300,2,0,49,Rental unit in Los Angeles,New,3,1,2
45577,1036526167744354319,189552555,Taynara,Pico-Robertson,Entire home/apt,101,30,0,89,Rental unit in Los Angeles,New,4,3,2
45586,1037575019510875191,72933825,The Maimon Group,Hollywood Hills West,Entire home/apt,936,31,0,177,Home in Los Angeles,New,3,3,4


In [79]:
print(airbnb_df.columns)

Index(['id', 'host_id', 'host_name', 'neighbourhood', 'room_type', 'price',
       'minimum_nights', 'number_of_reviews', 'availability_365',
       'airbnb_name', 'star', 'bedrooms', 'beds', 'baths'],
      dtype='object')


In [77]:
airbnb_df.to_csv('/content/drive/MyDrive/550 CrimeLA data/Airbnb.csv', index=False)